In [1]:
import re
import time
import json
import requests as r
from bs4 import BeautifulSoup as bs
from concurrent.futures import ThreadPoolExecutor, wait

In [2]:
def get_number():
    return _list.pop(0)

In [3]:
def worker(url):
    product = []
    pd_url = 'https://www.igift.tw' + url
    pd_resp = r.get(pd_url)
    pd_soup = bs(pd_resp.text, 'html5lib')
    pd_name = pd_soup.select_one("div > h1").text.strip()
    pd_price = pd_soup.select_one('div.price').text.strip()
    pd_description = pd_soup.select_one('div.description-container''').text.strip()

    jd = json.loads(re.findall("app.value\(\'product\', ({.*})\);", pd_resp.text)[0])
    pd_style = []
    picture_href = []
    for pic_url in jd['media']:
        picture_href.append(pic_url['default_image_url'])
        picture_href.append(pic_url['detail_image_url'])

    for style in jd['variations']:
        pd_style.append(style['fields'][0]["name"])

    product.append({
        'ID': 'lv' + "{:0>6d}".format(get_number()),
        'name': pd_name,
        'price': pd_price,
        'brand': 'IGIFT',
        'memo': pd_style,
        'desc': pd_description,
        'href': pd_url,
        'picture': picture_href,
        'category': "",
        'style': "",
        'comment': "",
        'othertag': ""
    })
    print("get product:" + url)

    load_to_csv(product)


In [4]:
def load_to_csv(product):
    with open("igift.json", "a") as f:
        jd = json.dumps(product, ensure_ascii=False, indent=4)
        jd.encode("utf-8")
        f.write(jd)

In [5]:
if __name__ == "__main__":
    tStart = time.time()
    url = 'https://www.igift.tw/products?page=%s'
    counter = 1
    switch = True
    urls = []
    _list = [i for i in range(10000)]
    while switch:
        resp = r.get(url % counter)
        soup = bs(resp.text, "html5lib")
        urls.extend([x.get('href') for x in soup.select('ul > li > a') if x.get('href').startswith('/products/')])
        pages = soup.select("ul.pagination > li > a")
        if pages:
            page = pages[-1].text
            if page == "»":
                print(counter)
                counter += 1
            else:
                for i in range(counter, int(page) + 1):
                    print(i)
                    resp = r.get(url % i)
                    soup = bs(resp.text, "html5lib")
                    urls.extend(
                        [x.get('href') for x in soup.select('ul > li > a') if x.get('href').startswith('/products/')])
                switch = False
        else:
            switch = False

    numThread = 80
    threads = ThreadPoolExecutor(numThread)
    futures = []
    for url in urls:
        futures.append(threads.submit(worker, url))
    wait(futures)

    tEnd = time.time()
    print(tEnd - tStart)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
get product:/products/5a1434f9080f066cc1000d8eget product:/products/59eda812492678b0be007203get product:/products/5a09654672fdc0e4e1000f33


get product:/products/59f975c54926784c200028b5get product:/products/5a0abfb672fdc051f6000571get product:/products/5a1bb9bc9a76f017d300001aget product:/products/5a0da1915521159bb40005e7get product:/products/59f6b7624e22a6ac80000158get product:/products/5a1fffb9080f06b9b00005a0



get product:/products/5a1ffe879f9a4f50eb000644

get product:/products/5a16e53f72fdc05c1a000bbaget product:/products/5a12e7d559563018160004e3

get product:/products/59fc09d459d5241802000679get product:/products/59edb1d572fdc087d600655e

get product:/products/5a02c0da9a76f0181c0002adget product:/products/59f815e6d4e3957a96000e1cget product:/products/59f5f61f595630181a0007f0
get product:/products/59e46b26552115fe77000226get product:/products/5a12a8b759d524cb04000875get product:/pro

get product:/products/58e9f65e72fdc07d6000dda1
get product:/products/58e7957e59d524172a00212d
get product:/products/58e0aa0259d5241736000174get product:/products/58e0a16cd4e395c868008063

get product:/products/58e0a3b9d4e3950c2e004275
get product:/products/58e7130c72fdc07d71003a45get product:/products/58be528e72fdc05a3f0033b5get product:/products/58d7d51c72fdc02f2a0160fdget product:/products/58e793abd4e3956735004fefget product:/products/58ca5c989a76f0122d000f26get product:/products/58d9e94f9a76f0173b000221get product:/products/58ea0d3a72fdc07d8f00ec40get product:/products/58bef7bd9a76f029f1001f73get product:/products/58dd1872115350176d0005a9get product:/products/58e0a6649a76f01711000ca0get product:/products/58cfa29f72fdc02c0e00636cget product:/products/58bc2e5d72fdc0a6fb00ba21get product:/products/58d2335a9a76f017620012a7get product:/products/58e48cfdd4e395328f005282get product:/products/58de9349d4e395c8750023e2get product:/products/58cf95a29a76f01733001411get product:/products/58de803

get product:/products/57f3437c617069b0c6720c00
get product:/products/57f4e02a617069efaeec0300get product:/products/5808491661706903e1ac0400

get product:/products/57c93d37617069731a5c3b00get product:/products/57e9146461706931485a0c00get product:/products/57e4339c6170690929611700


get product:/products/57c6e87c617069610f9b3600
get product:/products/57ecd6ee6170691765810600get product:/products/57c7fbab6170694f77f50900get product:/products/57d299276170695ca6cf4900get product:/products/57c3de916170696e79ac0900get product:/products/5795ff4c617069396dfd2700get product:/products/57937bae6170694c4f097200get product:/products/57c6f0d8617069116d260300get product:/products/57c305526170693437c04501get product:/products/57bf1d6e61706942bc708700get product:/products/57c7dee561706962da180300get product:/products/57c944d66170691158790f00get product:/products/57f930866170696d839f3d00get product:/products/57e96ee961706969a1011a00











get product:/products/57abf5046170695a42160000get product:/pr


get product:/products/56bdedfa69702d7ab5865f00
get product:/products/56a3b75769702d10562fa100get product:/products/56aa309869702d739a1d2a00
get product:/products/56af6b7769702d11345e2000get product:/products/56a74bdc69702d4ef9131100get product:/products/56a79cf169702d1383442800get product:/products/56af625169702d5ab10b1b00get product:/products/56c56acb61706913d2621600






get product:/products/56ac2bfd69702d7392ed9200get product:/products/56ac773169702d653fe7af00get product:/products/56be1c8869702d6b06806100get product:/products/56ac633469702d5d9cc0aa00




get product:/products/56a2664469702d104e895a00get product:/products/56b630fc69702d7467eef400get product:/products/56c5fc096170690d75d60a00get product:/products/56ab8ddb69702d7396178700get product:/products/56b779c769702d0c2bca2701get product:/products/569d011269702d43e9da7200get product:/products/56c682dc6170690d7d751b00get product:/products/56b3569c69702d746f3c7300get product:/products/569cfe6b69702d01e31a1500get product:/produc


get product:/products/55eeacfe69702d78ff7f0500get product:/products/55f1619069702d6a4c8d1900

get product:/products/55ee881b69702d2c2dbe0000get product:/products/55eea0a069702d2c311f0300get product:/products/55ee86d769702d71b3880000get product:/products/563b20be69702d51e8ff1e00get product:/products/55f1556969702d6fcf721700get product:/products/561b676969702d4d7bb15500get product:/products/55ee98c369702d2c33350200get product:/products/55efe46d69702d5a3c030000get product:/products/55ee9cbb69702d5c7b2a0300get product:/products/55ee830a69702d71ad600000get product:/products/55f1556969702d6fcf721700get product:/products/55ed7ca269702d2f81d60600get product:/products/55ee7cd269702d0b4ce31100get product:/products/55eea66769702d78ff590400get product:/products/55ee98c369702d2c33350200get product:/products/55ee93ea69702d203abf0100get product:/products/55ed5d0069702d2f7be60200




get product:/products/55eea66769702d78ff590400










get product:/products/55ed52b769702d77b7900100get product:/pr